In [9]:
import pandas as pd 
import numpy as np 
import json
from pymongo import MongoClient
import json

Setup MongoDB: https://zellwk.com/blog/install-mongodb/

### Erstelle Database

In [31]:
client = MongoClient('localhost', 27017)

In [32]:
db = client.Ecomerce
collection_amazon_fee = db.AmazonGebühren
collection_ppc_kosten = db.PpcGebühren
collection_lagergebühren = db.Lagergebühren
collection_sales = db.Sales
collection_dailyppc = db.DailyPpcCost

### Kosten Parser

Website to get the sell reviews: https://sellercentral.amazon.de/payments/reports/custom/request?tbla_daterangereportstable=sort:%7B%22sortOrder%22%3A%22DESCENDING%22%7D;search:undefined;pagination:1;

In [1]:
def cost_report_parser(file): 
    
    # Read Data
    df = pd.read_csv(file, skiprows = 6, decimal=',')  
    temporary_dict = df.to_dict("records")
    
    # Create lists for parsing
    customer_data = []
    werbekosten = []
    lagergebühr = []
    monatliche_gebühren = []
    
    """
    Export der Daten aus der CSV-Datei 

    """
    for element in temporary_dict: 
        # Create datetime stamp 
        tag = element['Datum/Uhrzeit'][0:2]
        monat = element['Datum/Uhrzeit'][3:5]
        jahr = element['Datum/Uhrzeit'][6:10]
        datum = jahr + "-" + monat + "-" + tag
        
        # Produktkosten
        if datum < "2020-09-08": 
            produktkosten = 3.35
        else: 
            produktkosten = 2.22

        """
        Verkaufte Einheiten

        """
        if element["Typ"] == "Bestellung":         
            bestellnummer = element["Bestellnummer"]       
            uhrzeit = element['Datum/Uhrzeit'][11:19]
            stunde = element['Datum/Uhrzeit'][11:13]
            abrechnungsnummer = element['Abrechnungsnummer']
            umsatz = element["Umsätze"]
            menge = element["Menge"]
            gewinn = element["Gesamt"]
            bundesland = element["Bundesland"]
            stadt = element['Ort der Bestellung']
            gutschrift_versandkosten = element['Gutschrift für Versandkosten']
            gutschrift_geschenkverpackung = element['Gutschrift für Geschenkverpackung']
            rabatte_werbekosten = element['Rabatte aus Werbeaktionen']
            verkaufsgebühren = element["Verkaufsgebühren"]
            gebühren_versand_durch_amazon = element['Gebühren zu Versand durch Amazon']
            andere_transaktionsgebühren = element['Andere Transaktionsgebühren']
            andere = element["Andere"]

            transaktionen = {"iD" : bestellnummer,
                       "datum" : datum,
                       "uhrzeit" : uhrzeit,
                       "stunde" : stunde,
                       "währung" : "€",
                       "umsatz" : umsatz,
                       "menge" : menge,
                       "gewinn" : round(gewinn, 2),
                       "produktkosten" : produktkosten * menge, 
                       "gewinn_nach_produktkosten" : round(float(gewinn) - (produktkosten * menge), 2),
                       "abrechnungsnummer" : abrechnungsnummer, 
                       "geodata" : {"bundesland" : bundesland, "stadt" : stadt},
                       "kostenschlüssel" : {"gutschrift_versandkosten" : gutschrift_versandkosten,
                                            "gutschrift_geschenkverpackung" : gutschrift_geschenkverpackung,
                                            "rabatte_werbekosten" : rabatte_werbekosten,
                                            "verkaufsgebühren" : verkaufsgebühren,
                                            "gebühren_versand_durch_amazon" : gebühren_versand_durch_amazon,
                                            "andere_transaktionsgebühren" : andere_transaktionsgebühren,
                                            "andere" : andere 
                                           }
                         }


            customer_data.append(transaktionen)


        """
        Werbekosten

        """
        if element["Typ"] == 'Servicegebühr':
            if element["Beschreibung"] == "Werbekosten":
                uhrzeit = element['Datum/Uhrzeit'][11:19]
                kosten = element["Gesamt"]
                abrechnungsnummer = element["Abrechnungsnummer"]

                transaktionen_2 = {"iD" : abrechnungsnummer,
                                   "datum" : datum,
                                   "uhrzeit" : uhrzeit, 
                                   "währung" : "€",
                                   "kosten" : kosten,
                                   }          
                werbekosten.append(transaktionen_2)

        """
        Lagergebühr

        """    
        if element["Typ"] == 'Versand durch Amazon Lagergebühr': 
            uhrzeit = element['Datum/Uhrzeit'][11:19]
            kosten = element["Gesamt"]
            abrechnungsnummer = element["Abrechnungsnummer"]

            transaktionen_3 = {"iD" : abrechnungsnummer,
                               "datum" : datum,
                               "uhrzeit" : uhrzeit, 
                               "währung" : "€",
                               "kosten" : kosten,
                               }          
            lagergebühr.append(transaktionen_3)


        """
        Monatliche Amazon Gebühren 

        """
        if element["Typ"] == "Servicegebühr": 
            if element["Beschreibung"] == 'SellerPayments_Report_Fee_Subscription':
                uhrzeit = element['Datum/Uhrzeit'][11:19]
                kosten = element["Gesamt"]
                abrechnungsnummer = element["Abrechnungsnummer"]

                transaktionen_4 = {"iD" : abrechnungsnummer,
                                   "datum" : datum,
                                   "uhrzeit" : uhrzeit,
                                   "währung" : "€",
                                   "umsatz" : kosten
                                   }          
                monatliche_gebühren.append(transaktionen_4)
    
    return customer_data, werbekosten, lagergebühr, monatliche_gebühren

In [5]:
file = r'2020Apr.1-2020Okt.10CustomTransaction.csv'

In [37]:
customer_data, werbekosten, lagergebühr, monatliche_gebühren = cost_report_parser(file)

### Tägliche PPC Kosten Parser

Website to get the sell reviews: https://sellercentral.amazon.de/payments/reports/custom/request?tbla_daterangereportstable=sort:%7B%22sortOrder%22%3A%22DESCENDING%22%7D;search:undefined;pagination:1;

In [38]:
def daily_ppc_cost(file): 
    df_2 = pd.read_excel(file_2)  
    temporary_dict_2 = df_2.to_dict("records")
    daily_ppc = []
    
    for element in temporary_dict_2: 
        datum = str(element["Datum"])[:10]
        klicks = element["Klicks"]
        kosten_pro_klick = element["Kosten pro Klick (CPC)"]
        ausgaben = element["Ausgaben"]

        daily_ppc_cost = {
                       "datum" : datum,
                       "klicks" : klicks, 
                       "kosten_pro_klick" : round(kosten_pro_klick, 2),
                       "ausgaben" : round(ausgaben, 2), 
                       "währung" : "€"

                           }          
        daily_ppc.append(daily_ppc_cost)
    return daily_ppc

In [39]:
file_2 = r'/Users/jonasbirkendahl/Downloads/Sponsored Products Leistung im Zeitabschnitt report.xlsx'

In [41]:
daily_ppc = daily_ppc_cost(file_2)

### Daten in Kollektionen einfügen

In [42]:
collection_sales.insert_many(customer_data)

In [43]:
collection_ppc_kosten.insert_many(werbekosten)

In [44]:
collection_lagergebühren.insert_many(lagergebühr)

In [45]:
collection_amazon_fee.insert_many(monatliche_gebühren)

In [46]:
collection_dailyppc.insert_many(daily_ppc)